<a href="https://colab.research.google.com/github/realldy/MachineLearning/blob/master/NNPSO51_ipynb%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#结合梯度下降和动量的粒子群算法+eta3优化+eta1优化+without gamma
import random
random.seed(1) 
import numpy as np
np.random.seed(1)
from scipy import special
from scipy import stats
import math

In [2]:
train_x = [random.uniform(0, 4) for i in range(5000)]

train_x = np.array(train_x, dtype=np.float64) 
print(train_x)

[0.53745698 3.38973495 3.05509848 ... 3.816279   3.79124414 1.83822385]


In [3]:
def f1(train_x,lamda):
  return 1-np.exp(-np.multiply(lamda,train_x))

In [4]:
def f2(train_x,mu1,s1):
  erf1 = special.erf(np.divmod(train_x-mu1,np.multiply(s1,np.math.sqrt(2))))
  return 0.5*(1+erf1)

In [5]:
def f3(train_x,mu2,s2):
  erf2 = special.erf(np.divmod(np.log(train_x)-mu2,np.multiply(s2,np.math.sqrt(2))))
  return 0.5*(1+erf2)

In [6]:
def fit(x,lamda,mu1,s1,mu2,s2,w1,w2,w3):
  y_pre = (w1*f1(x,lamda)+w2*f2(x,mu1,s1)+w3*f3(x,mu2,s2))/(w1+w2+w3)
  return y_pre

In [7]:
#def f4(train_x,alpha,beta):
#  return statis.gamma.cdf(train_x,alpha,0,beta)

In [8]:
train_y = f1(train_x,0.5)

In [9]:
def function(lamda,mu1,s1,mu2,s2,w1,w2,w3):
  y_pre = (w1*f1(train_x,lamda)+w2*f2(train_x,mu1,s1)+w3*f3(train_x,mu2,s2))/(w1+w2+w3)
  loss = np.mean(0.5*(y_pre-train_y)**2)
  return loss  

In [10]:
def fit1(x,part):
  y_pre = (part[5]*f1(x,part[0])+part[6]*f2(x,part[1],part[2])+part[7]*f3(x,part[3],part[4]))/(part[5]+part[6]+part[7])
  return y_pre


In [11]:
class Particle:
    # 初始化
    def __init__(self, x_max, max_vel, dim):
        self.__pos = [random.uniform(-x_max, x_max) for i in range(dim)]  # 粒子的位置
        self.__vel = [random.uniform(-max_vel, max_vel) for i in range(dim)]  # 粒子的速度
        self.__bestPos = [0.0 for i in range(dim)]  # 粒子最好的位置
        self.__fitnessValue = function(self.__pos[0],self.__pos[1],self.__pos[2],self.__pos[3],self.__pos[4],self.__pos[5],self.__pos[6],self.__pos[7])  # 适应度函数值
    def set_pos(self, i, value):
        self.__pos[i] = value

    def get_pos(self):
        return self.__pos

    def set_best_pos(self, i, value):
        self.__bestPos[i] = value

    def get_best_pos(self):
        return self.__bestPos

    def set_vel(self, i, value):
        self.__vel[i] = value

    def get_vel(self):
        return self.__vel

    def set_gradient(self, i, value):
        self.__gradient[i] = value
    
    def get_gradient(self):
        return self.__gradient

    def get_weight_gradient(self):
        return self.weight_gradient

    def set_weight_gradient(self, i, value):
        self.weight_gradient[i] = value

    def set_eta1(self, i, value):
        self.eta1[i] = value

    def get_eta1(self):
        return self.eta1


    def set_fitness_value(self, value):
        self.__fitnessValue = value

    def get_fitness_value(self):
        return self.__fitnessValue 

In [12]:
class PSO:
    def __init__(self, dim, size, iter_num, x_max, max_vel, best_fitness_value=float('Inf'), C1=2, C2=2,  W=1):
        self.C1 = C1
        self.C2 = C2
        self.W = W
        self.dim = dim  # 粒子的维度
        self.size = size  # 粒子个数
        self.iter_num = iter_num  # 迭代次数
        self.x_max = x_max
        self.max_vel = max_vel  # 粒子最大速度
        self.best_fitness_value = best_fitness_value
        self.best_position = [random.randint(1,3) for i in range(dim)]  # 种群最优位置
        self.fitness_val_list = []  # 每次迭代最优适应值

        # 对种群进行初始化
        self.Particle_list = [Particle(self.x_max, self.max_vel, self.dim) for i in range(self.size)]
    def set_bestFitnessValue(self, value):
        self.best_fitness_value = value

    def get_bestFitnessValue(self):
        return self.best_fitness_value

    def set_bestPosition(self, i, value):
        self.best_position[i] = value

    def get_bestPosition(self):
        return self.best_position

    # 更新速度
    def update_vel(self, part):
        for i in range(self.dim):
            vel_value = self.W * part.get_vel()[i] + self.C1  * random.random() * (part.get_best_pos()[i] - part.get_pos()[i]) + self.C2 * random.random() * (self.get_bestPosition()[i] - part.get_pos()[i])
            if vel_value > self.max_vel:
                vel_value = self.max_vel
            elif vel_value < -self.max_vel:
                vel_value = -self.max_vel
            part.set_vel(i, vel_value)
    
    

    # 更新位置
    def update_pos(self, part):
        for i in range(self.dim):
            pos_value = part.get_pos()[i] + part.get_vel()[i]
            if i==8:
              if pos_value>0:
                part.set_pos(i, pos_value)
            else:
              part.set_pos(i, pos_value)
        value = function(part.get_pos()[0],part.get_pos()[1],part.get_pos()[2],part.get_pos()[3],part.get_pos()[4],part.get_pos()[5],part.get_pos()[6],part.get_pos()[7])
        if value < part.get_fitness_value():
            part.set_best_pos(i, part.get_pos()[i])
        if value < self.get_bestFitnessValue():
            self.set_bestFitnessValue(value)
            for i in range(self.dim):
                self.set_bestPosition(i, part.get_pos()[i])

    def update(self):
        for i in range(self.iter_num):
            for part in self.Particle_list:
                self.update_vel(part)  # 更新速度
                self.update_pos(part)  # 更新位置
                
            self.fitness_val_list.append(self.get_bestFitnessValue())  # 每次迭代完把当前的最优适应度存到列表
            #print("PSO当前最优位置:"+str(self.get_bestPosition()))
            print("迭代:"+str(i))
            print("PSO解:"+str(self.get_bestFitnessValue()))
        return self.fitness_val_list, self.get_bestPosition()

In [33]:
dim = 8
size = 100
iter_num = 1000
x_max = 100
max_vel = 0.5

pso = PSO(dim, size, iter_num, x_max, max_vel)
fit_var_list1, best_pos1 = pso.update()
print("PSO最优位置:" + str(best_pos1))
print("PSO最优解:" + str(fit_var_list1[-1]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in square
  This is separate from the ipykernel package so we can avoid doing imports until


迭代:0
PSO解:0.03064429519687353
迭代:1
PSO解:0.03063020376242324
迭代:2
PSO解:0.0306159233516139
迭代:3
PSO解:0.030601450638691817
迭代:4
PSO解:0.03058678224687575
迭代:5
PSO解:0.030571914749607316


/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:160: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


迭代:6
PSO解:0.02994258707578892
迭代:7
PSO解:0.02994258707578892
迭代:8
PSO解:0.02994258707578892
迭代:9
PSO解:0.02994258707578892
迭代:10
PSO解:0.02994258707578892
迭代:11
PSO解:0.02994258707578892
迭代:12
PSO解:0.02994258707578892
迭代:13
PSO解:0.02994258707578892
迭代:14
PSO解:0.02994258707578892
迭代:15
PSO解:0.02994258707578892
迭代:16
PSO解:0.02994258707578892
迭代:17
PSO解:0.02994258707578892
迭代:18
PSO解:0.02994258707578892
迭代:19
PSO解:0.02994258707578892
迭代:20
PSO解:0.028706642587208238
迭代:21
PSO解:0.028706642587208238
迭代:22
PSO解:0.028706642587208238
迭代:23
PSO解:0.028294456208392285
迭代:24
PSO解:0.02639369717017015
迭代:25
PSO解:0.02639369717017015
迭代:26
PSO解:0.02639369717017015
迭代:27
PSO解:0.02639369717017015
迭代:28
PSO解:0.02639369717017015
迭代:29
PSO解:0.02639369717017015
迭代:30
PSO解:0.02639369717017015
迭代:31
PSO解:0.02639369717017015
迭代:32
PSO解:0.02639369717017015
迭代:33
PSO解:0.022744179392590696
迭代:34
PSO解:0.020743550548441892
迭代:35
PSO解:0.020743550548441892
迭代:36
PSO解:0.014021761613973758
迭代:37
PSO解:0.014021761613973758
迭代:

In [14]:
def MAPE(y_pre,y_true):
  return np.mean(np.absolute((y_pre - y_true)/y_true))*100

In [36]:
test_x = [random.uniform(0, 4) for i in range(100)]
test_x = np.array(test_x, dtype=np.float64) 
y_true = f1(test_x,0.5)
y_pre = fit(test_x,0.49959442589047254, 0.06682115269763406, -1.6465578102674971, 1.7409543396058103, 0.3336867190655697, 2.5201122581073543, 0.00505018348526462, -0.008365427270166897)
print(test_x)
print(y_pre)
print(y_true)
print(MAPE(y_pre,y_true))

[2.45021262 1.55072587 3.82439524 3.11747605 3.44978045 3.46693816
 0.67498383 1.99125468 2.44126977 1.0132305  3.06216036 3.33899361
 2.65886521 0.92692495 3.76147057 0.20082575 1.59163014 1.56255906
 1.06092871 1.56736904 3.93495392 3.74998453 1.18999838 3.67369358
 1.31349026 0.52541766 0.19310993 2.34917749 2.06526364 0.35065218
 3.65185024 0.63137253 0.86455698 3.87821391 0.63492757 1.00504031
 1.31220275 2.92506509 2.66027065 3.37175238 2.87568346 2.16557387
 3.80928228 2.34647597 1.04681283 0.06134786 0.5159179  2.77685067
 0.55023704 1.3994286  1.44005868 3.24939512 0.34914653 0.7619091
 1.51186456 1.87035864 0.79337228 1.35457985 2.45142882 0.14610196
 2.4225774  3.88790011 0.99150607 3.61109214 0.52450993 2.29773142
 3.58489668 2.01061108 2.54944113 3.63431852 0.27322611 2.24573206
 3.16362993 3.63816149 1.85873066 1.40741    3.33355306 0.67754783
 2.71124925 2.32149649 3.91757166 3.53609652 1.82280031 0.01567457
 0.72844535 1.13704768 0.83141184 2.20252147 1.78193625 3.85770

In [16]:
def pdf(x,lamda,mu1,s1,mu2,s2,w1,w2,w3):
  p1 = lamda*np.exp(-lamda*x)
  p2 = np.exp(-np.square(x-mu1)/(2*np.square(s1))) / (np.sqrt(x*np.pi)*s1)
  p3 = np.exp(-np.square(np.log(x)-mu2) / (2*np.square(s2))) / (np.sqrt(x*np.pi)*s2*x)
  return (w1*p1+w2*p2+w3*p3)/(w1+w2+w3)

In [37]:
pdf_true = 0.5*np.exp(-0.5*test_x)
pdf_pre = pdf(test_x,0.49959442589047254, 0.06682115269763406, -1.6465578102674971, 1.7409543396058103, 0.3336867190655697, 2.5201122581073543, 0.00505018348526462, -0.008365427270166897)
print(pdf_true)
print(pdf_pre)
print(MAPE(pdf_pre,pdf_true))

[0.14686324 0.2302683  0.07387766 0.10520071 0.08909631 0.08833523
 0.35677887 0.18474579 0.14752139 0.30126578 0.10815095 0.09417091
 0.13231368 0.31455081 0.07623898 0.45223195 0.22560666 0.22890992
 0.29416586 0.22836006 0.06990458 0.07667808 0.27578151 0.0796595
 0.25926818 0.38448288 0.45397999 0.154473   0.17803431 0.41959166
 0.08053428 0.36464411 0.3245143  0.07191617 0.36399652 0.30250202
 0.25943514 0.11582444 0.13222074 0.09264101 0.11871983 0.16932521
 0.07443803 0.1546818  0.2962494  0.48489587 0.38631347 0.12473391
 0.37974105 0.2483636  0.24336899 0.09848562 0.41990766 0.34160447
 0.23478632 0.19625774 0.33627254 0.25399591 0.14677396 0.46477672
 0.14890662 0.07156872 0.30455603 0.08219233 0.38465743 0.15849807
 0.08327595 0.18296641 0.13975453 0.08124334 0.43615385 0.162673
 0.1028008  0.08108738 0.1974021  0.24737443 0.09442743 0.35632177
 0.12889311 0.15662585 0.07051478 0.08533288 0.20098051 0.49609667
 0.34736824 0.28318043 0.32993715 0.16622584 0.20512919 0.0726573